# Estimating Non-Mandatory Tour Frequency

This notebook illustrates how to re-estimate a single model component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [ ]:
import larch as lx
import pandas as pd

lx.versions()

JAX not found. Some functionality will be unavailable.


{'larch': '6.0.32',
 'sharrow': '2.13.0',
 'numpy': '1.26.4',
 'pandas': '1.5.3',
 'xarray': '2024.3.0',
 'numba': '0.60.0'}

For this demo, we will assume that you have already run ActivitySim in estimation
mode, and saved the required estimation data bundles (EDB's) to disk.  See
the [first notebook](./01_estimation_mode.ipynb) for details.  The following module
will run a script to set everything up if the example data is not already available.

In [2]:
from est_mode_setup import prepare

prepare()

EDB directory already populated.


PosixPath('test-estimation-data/activitysim-prototype-mtc-extended')

# Load data and prep model for estimation

In [3]:
modelname = "nonmand_tour_freq"

from activitysim.estimation.larch import component_model

model, data = component_model(
    modelname,
    edb_directory=f"output-est-mode/estimation_data_bundle/non_mandatory_tour_frequency/",
    return_data=True,
    condense_parameters=True,
)

Loading EDB for PTYPE_FULL segment
loading from output-est-mode/estimation_data_bundle/non_mandatory_tour_frequency/PTYPE_FULL/non_mandatory_tour_frequency_coefficients_PTYPE_FULL.csv
loading from output-est-mode/estimation_data_bundle/non_mandatory_tour_frequency/PTYPE_FULL/non_mandatory_tour_frequency_choosers_combined.parquet
loading from output-est-mode/estimation_data_bundle/non_mandatory_tour_frequency/PTYPE_FULL/non_mandatory_tour_frequency_interaction_expression_values.parquet
Loading EDB for PTYPE_PART segment
loading from output-est-mode/estimation_data_bundle/non_mandatory_tour_frequency/PTYPE_PART/non_mandatory_tour_frequency_coefficients_PTYPE_PART.csv
loading from output-est-mode/estimation_data_bundle/non_mandatory_tour_frequency/PTYPE_PART/non_mandatory_tour_frequency_choosers_combined.parquet
loading from output-est-mode/estimation_data_bundle/non_mandatory_tour_frequency/PTYPE_PART/non_mandatory_tour_frequency_interaction_expression_values.parquet
Loading EDB for PTYP

The prototype model spec we are re-estimating has 210 rows for each person type, but the
accompanying dataset is not large enough to successfully estimate anywhere near than many
parameters. The `condense_parameters` option is activated here as a short cut to making
a model that can be estimated with stable parameter results.  When activated, it merges
parameters not only by name (i.e. when the same name appears twice it is the same parameter)
but also by value, so that if the initial value of any two parameters is identical
then they are treated as the same parameter.  Using "condense_parameters" in actual model
estimation efforts is ill advised and may generate confusing or unexpected results.

This component actually has a distinct choice model for each person type, so
instead of a single model there's a `dict` of models.

In [4]:
type(model)

dict

In [5]:
model.keys()

dict_keys(['PTYPE_FULL', 'PTYPE_PART', 'PTYPE_UNIVERSITY', 'PTYPE_NONWORK', 'PTYPE_RETIRED', 'PTYPE_DRIVING', 'PTYPE_SCHOOL', 'PTYPE_PRESCHOOL'])

# Review data loaded from the EDB

We can review the data loaded as well, similarly there is seperate data 
for each person type.

## Coefficients

In [6]:
data.coefficients['PTYPE_FULL']

,value,constrain
coefficient_name,,
coef_escorting_tour,0.0000,T
coef_discretionary_tour,0.0000,T
coef_shopping_tour,0.0000,T
coef_maintenance_tour,0.0000,T
coef_visiting_or_social_tour,0.0000,T
...,...,...
coef_1_plus_maintenance_tours_constant,0.1202,F
coef_1_plus_eating_out_tours_constant,0.0097,F
coef_1_plus_visting_tours_constant,0.0522,F


## Utility specification

In [7]:
data.spec['PTYPE_FULL']

0                   coef_urban_and_discretionary_tour
1                   coef_urban_and_discretionary_tour
2                   coef_urban_and_discretionary_tour
3                   coef_urban_and_discretionary_tour
4                   coef_urban_and_discretionary_tour
                            ...                      
205            coef_1_plus_maintenance_tours_constant
206             coef_1_plus_eating_out_tours_constant
207                coef_1_plus_visting_tours_constant
208    coef_1_plus_other_discretionary_tours_constant
209          coef_0_auto_household_and_escorting_tour
Name: PTYPE_FULL, Length: 210, dtype: object

## Chooser data

In [8]:
data.chooser_data['PTYPE_FULL']

,person_id,model_choice,override_choice,household_id,age,PNUM,sex,pemploy,pstudent,ptype,...,high_income,no_cars,car_sufficiency,num_hh_joint_shop_tours,num_hh_joint_eatout_tours,num_hh_joint_maint_tours,num_hh_joint_social_tours,num_hh_joint_othdiscr_tours,has_mandatory_tour,has_joint_tour
0,72355,0,0,72355,52,1,2,1,3,1,...,False,False,1,0,0,0,0,0,1,0
1,72384,0,0,72384,28,1,2,1,3,1,...,False,False,0,0,0,0,0,0,1,0
2,72407,1,0,72407,52,1,2,1,3,1,...,False,False,0,0,0,0,0,0,1,0
3,72459,0,0,72459,38,1,1,1,3,1,...,False,True,-1,0,0,0,0,0,1,0
4,72569,1,20,72569,49,1,1,1,3,1,...,False,True,-1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19354,7538573,0,0,2847238,67,1,2,1,3,1,...,False,True,-1,0,0,0,0,0,1,0
19355,7538990,1,19,2847655,72,1,1,1,3,1,...,False,True,-1,0,0,0,0,0,0,0
19356,7539071,0,0,2847736,70,1,1,1,3,1,...,False,False,1,0,0,0,0,0,1,0
19357,7539203,0,0,2847868,87,1,1,1,3,1,...,False,False,0,0,0,0,0,0,1,0


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  The prototype model spec we are re-estimating has 210 rows for each person type, but the accompanying dataset is not large enough to successfully estimate anywhere near than many parameters, so a short cut is applied by having one parameter only per unique existing parameter value.

In [9]:
model['PTYPE_FULL'].datatree.root_dataset

<xarray.Dataset> Size: 2GB
Dimensions:                                                                      (
                                                                                  person_id: 19359,
                                                                                  alt_id: 96)
Coordinates:
  * person_id                                                                    (person_id) int64 155kB ...
  * alt_id                                                                       (alt_id) int64 768B ...
Data variables: (12/353)
    index                                                                        (person_id, alt_id) int64 15MB ...
    util_escorting_tour                                                          (person_id, alt_id) int64 15MB ...
    util_discretionary_tour                                                      (person_id, alt_id) int64 15MB ...
    util_shopping_tour                                                           (person_id, alt_id) int64 15MB ...
    util_maintenance_tour                                                        (person_id, alt_id) int64 15MB ...
    util_visiting_or_social_tour                                                 (person_id, alt_id) int64 15MB ...
    ...                                                                           ...
    num_hh_joint_eatout_tours                                                    (person_id) int8 19kB ...
    num_hh_joint_maint_tours                                                     (person_id) int8 19kB ...
    num_hh_joint_social_tours                                                    (person_id) int8 19kB ...
    num_hh_joint_othdiscr_tours                                                  (person_id) int8 19kB ...
    has_mandatory_tour                                                           (person_id) int64 155kB ...
    has_joint_tour                                                               (person_id) int64 155kB ...
Attributes:
    _caseid_:  person_id
    _altid_:   alt_id

In [10]:
for k, m in model.items():
    print(f"Person type {k} has {len(m.utility_ca)} utility terms and {len(m.pf)} unique parameters.")

Person type PTYPE_FULL has 210 utility terms and 72 unique parameters.
Person type PTYPE_PART has 210 utility terms and 51 unique parameters.
Person type PTYPE_UNIVERSITY has 210 utility terms and 70 unique parameters.
Person type PTYPE_NONWORK has 210 utility terms and 77 unique parameters.
Person type PTYPE_RETIRED has 210 utility terms and 53 unique parameters.
Person type PTYPE_DRIVING has 210 utility terms and 43 unique parameters.
Person type PTYPE_SCHOOL has 210 utility terms and 34 unique parameters.
Person type PTYPE_PRESCHOOL has 210 utility terms and 25 unique parameters.


For future estimation work, parameters can be intelligently named and applied to match the model developer's desired structure (by using the same named parameter for multiple rows of the spec file).  If this is done, the "short cut" should be disabled by setting `condense_parameters=False` in the loading step above.

Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [11]:
for k, m in model.items():
    print(f"Model {k}")
    m.set_cap(20)
    m.estimate(method='SLSQP', maxiter=900)

Model PTYPE_FULL


,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
coef_0_auto_household_and_escorting_tour,-2.000000,-2.000000,-2.0000,-2.0,-2.0,0.0,1
coef_1_escort_tour_constant,-0.065728,-0.065728,0.0298,-20.0,20.0,0.0,0
coef_1_plus_eating_out_tours_constant,0.047768,0.047768,0.0097,-20.0,20.0,0.0,0
coef_1_plus_maintenance_tours_constant,0.079776,0.079776,0.1202,-20.0,20.0,0.0,0
coef_1_plus_other_discretionary_tours_constant,0.733799,0.733799,0.7412,-20.0,20.0,0.0,0
...,...,...,...,...,...,...,...
coef_walk_access_to_retail_and_discretionary,0.060958,0.060958,0.0567,-20.0,20.0,0.0,0
coef_walk_access_to_retail_and_eating_out,0.156964,0.156964,0.1450,-20.0,20.0,0.0,0
coef_walk_access_to_retail_and_escorting,0.066031,0.066031,0.0451,-20.0,20.0,0.0,0


/Users/jpn/Git/est-mode/larch/src/larch/model/jaxmodel.py:1156: PossibleOverspecification: Model is possibly over-specified (hessian is nearly singular).
  self.calculate_parameter_covariance()


Model PTYPE_PART


,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
coef_0_auto_household_and_escorting_tour,-2.000000,-2.000000,-2.000000,-2.0,-2.0,0.0,1
coef_1_escort_tour_constant,0.430423,0.430423,0.527200,-20.0,20.0,0.0,0
coef_1_plus_eating_out_tours_constant,0.732307,0.732307,0.691400,-20.0,20.0,0.0,0
coef_1_plus_maintenance_tours_constant,0.480847,0.480847,0.553300,-20.0,20.0,0.0,0
coef_1_plus_other_discretionary_tours_constant,0.900550,0.900550,0.798900,-20.0,20.0,0.0,0
coef_1_plus_shopping_tours_constant,0.731723,0.731723,0.756900,-20.0,20.0,0.0,0
coef_1_plus_visting_tours_constant,0.250360,0.250360,0.140500,-20.0,20.0,0.0,0
coef_2_plus_escort_tours_constant,1.457716,1.457716,1.598700,-20.0,20.0,0.0,0
coef_car_shortage_vs_workers_and_tour_frequency_is_5_plus,-0.400969,-0.400969,-0.549800,-20.0,20.0,0.0,0


...

,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
coef_0_auto_household_and_escorting_tour,-2.000000,-2.000000,-2.0000,-2.0,-2.0,0.0,1
coef_1_escort_tour_constant,-0.710427,-0.710427,-0.7551,-20.0,20.0,0.0,0
coef_1_plus_eating_out_tours_constant,1.267594,1.267594,1.1145,-20.0,20.0,0.0,0
coef_1_plus_maintenance_tours_constant,-0.580534,-0.580534,-0.5060,-20.0,20.0,0.0,0
coef_1_plus_other_discretionary_tours_constant,0.614538,0.614538,0.4634,-20.0,20.0,0.0,0
coef_1_plus_shopping_tours_constant,1.037830,1.037830,0.4783,-20.0,20.0,0.0,0
coef_1_plus_visting_tours_constant,-0.445905,-0.445905,-0.4006,-20.0,20.0,0.0,0
coef_2_plus_escort_tours_constant,-0.076053,-0.076053,-0.0086,-20.0,20.0,0.0,0
coef_auto_access_to_retail_and_escorting,0.063236,0.063236,0.0629,-20.0,20.0,0.0,0


/Users/jpn/Git/est-mode/larch/src/larch/model/jaxmodel.py:1156: PossibleOverspecification: Model is possibly over-specified (hessian is nearly singular).
  self.calculate_parameter_covariance()


Model PTYPE_PRESCHOOL


,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
coef_0_auto_household_and_escorting_tour,-2.000000,-2.000000,-2.000000,-2.0,-2.0,0.0,1
coef_1_escort_tour_constant,0.274841,0.274841,0.362200,-20.0,20.0,0.0,0
coef_1_plus_eating_out_tours_constant,-0.253387,-0.253387,0.961200,-20.0,20.0,0.0,0
coef_1_plus_maintenance_tours_constant,0.316648,0.316648,0.678800,-20.0,20.0,0.0,0
coef_1_plus_other_discretionary_tours_constant,1.168896,1.168896,1.493500,-20.0,20.0,0.0,0
coef_1_plus_shopping_tours_constant,1.139680,1.139680,1.691900,-20.0,20.0,0.0,0
coef_1_plus_visting_tours_constant,0.059049,0.059049,0.442400,-20.0,20.0,0.0,0
coef_2_plus_escort_tours_constant,1.941139,1.941139,2.221900,-20.0,20.0,0.0,0
coef_discretionary_tour,0.578396,0.578396,0.903000,-20.0,20.0,0.0,0


/Users/jpn/Git/est-mode/larch/src/larch/model/jaxmodel.py:1156: PossibleOverspecification: Model is possibly over-specified (hessian is nearly singular).
  self.calculate_parameter_covariance()


### Estimated coefficients

In [12]:
model['PTYPE_FULL'].parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value,Constrained
Parameter,,,,,,
coef_0_auto_household_and_escorting_tour,-2.00,0.00,NA,,0.00,fixed value
coef_1_escort_tour_constant,-0.0657,49.6,-0.00,,0.00,
coef_1_plus_eating_out_tours_constant,0.0478,49.6,0.00,,0.00,
coef_1_plus_maintenance_tours_constant,0.0798,49.6,0.00,,0.00,
coef_1_plus_other_discretionary_tours_constant,0.734,49.6,0.01,,0.00,
coef_1_plus_shopping_tours_constant,0.360,49.6,0.01,,0.00,
coef_1_plus_visting_tours_constant,-0.0190,49.6,-0.00,,0.00,
coef_2_plus_escort_tours_constant,0.688,99.2,0.01,,0.00,
coef_at_home_pre_driving_school_kid_and_escorting_tour,-0.471,0.160,-2.94,**,0.00,


# Output Estimation Results

In [13]:
from activitysim.estimation.larch import update_coefficients
for k, m in model.items():
    result_dir = data.edb_directory/k/"estimated"
    update_coefficients(
        m, data.coefficients[k], result_dir,
        output_file=f"{modelname}_{k}_coefficients_revised.csv",
        relabel_coef=data.relabel_coef.get(k),
    );

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [14]:
for k, m in model.items():
    result_dir = data.edb_directory/k/"estimated"
    m.to_xlsx(
        result_dir/f"{modelname}_{k}_model_estimation.xlsx", 
        data_statistics=False,
    )

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [15]:
result_dir = data.edb_directory/'PTYPE_FULL'/"estimated"
pd.read_csv(result_dir/f"{modelname}_PTYPE_FULL_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_escorting_tour,0.000000,T
1,coef_discretionary_tour,0.000000,T
2,coef_shopping_tour,0.000000,T
3,coef_maintenance_tour,0.000000,T
4,coef_visiting_or_social_tour,0.000000,T
...,...,...,...
205,coef_1_plus_maintenance_tours_constant,0.079776,F
206,coef_1_plus_eating_out_tours_constant,0.047768,F
207,coef_1_plus_visting_tours_constant,-0.019022,F
208,coef_1_plus_other_discretionary_tours_constant,0.733799,F
